In [2]:
import pandas as pd
df = pd.read_csv("../data/raw/main_report.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [16]:
df.columns

Index(['email', 'can2_user_tags', 'uuid', 'can2_lifetime_value'], dtype='object')

In [52]:
# df[df["can2_user_tags"].str.contains("Possibly_Reform", na=False)]
# # or with loc


df.loc[df["can2_user_tags"].str.contains("Possibly_Green", na=False)]

,email,can2_user_tags,uuid,can2_lifetime_value
61135,benofficial@gmail.com,"Admin / Ladder / Completed Welcome Ladder, Don...",c66f36e4-76a3-4f12-9ca3-cf6d70c381bb,5
62146,jj@4ard.co.uk,"Action / 2023 STS / Attended, Action / 2023 ST...",49ce3b6b-cce4-4b57-a131-8a5a79efc0fb,0


In [56]:
test_list = df.loc[df["email"] == "roland.hamblin@btinternet.com", "can2_user_tags"].tolist()
test_list

['Admin / Ladder / Completed Welcome Ladder, Christmas Appeal Letter 2025, Donor, Donor_Crowdfunder2025Summer_Donor, Oct2025_anti-farage, SmartRaise']